In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.special import zeta
from mpmath import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Trials

In [ ]:
# Rienmann zeta trial - Gamma(2)=pi^2/6
zeta(2), (np.pi**2)/6

(mpf('1.6449340668482264'), 1.6449340668482264)

In [ ]:
# Polylogarithm
polylog(1,0.5), np.log(2), type(polylog(1,0.5))

(mpf('0.69314718055994529'), 0.6931471805599453, mpmath.ctx_mp_python.mpf)

In [ ]:
polylog(3, exp(2*pi*j/3))  # (-0.534247512515375 + 0.765587078525922j)

mpc(real='-0.53424751251537506', imag='0.76558707852592156')

In [ ]:
# mpc/mpf types conversion
ccc = polylog(2, exp(-j*pi))
ccc, complex(ccc)

(mpc(real='-0.8224670334241132', imag='-8.4350929019372245e-17'),
 (-0.8224670334241132-8.435092901937224e-17j))

## Paper approach

In [2]:
# Change l0, R0, mu and m - Other parameters always fixed         mu=0.2    l0=3    R0=14.14
l0 = 3 #
R0 = 14.14 #
mu = 0.2 #
lambd0 = 1 #
m = 15/R0


M =  400                    # (M+1)**2 is the number of patches or (2M+1)**2?
N = 2*M*l0 + 20*M*l0        # 2*M*l0 + 20*M*l0 + 1
N_p = (N-1)//2
N, N_p, 2*np.pi/N

(26400, 13199, 0.0002379994434537722)

In [3]:
def p_k(m, k):
  norm = ((np.e)**(1/m) + 1) / ((np.e)**(1/m) - 1)
  num = ( np.sinh(1/m) ) / ( np.cosh(1/m) - np.cos(k) )
  return num/norm

### Even function + Symmetries

In [4]:
I = 0

for x in tqdm(range(0, N_p+1)):
    for y in range(0, x+1):

      if x==0 and y==0:
        p_kx = p_k(m, 2*np.pi*x/N)
        I += (2*M+1)**2 / (lambd0*(1 -  p_kx*p_kx ) + mu)


      elif x!=0 and y==0:
        p_kx, p_ky = p_k(m, 2*np.pi*x/N), p_k(m, 2*np.pi*y/N)
        denom = lambd0*(1 -  p_kx*p_ky ) + mu

        numer_x = np.sin( (2*M+1)*np.pi*l0*x/N )
        denom_x = np.sin( np.pi*l0*x/N )

        I += 4*((2*M+1)*numer_x)/(denom*denom_x)


      elif x==y and x!=0 and y!=0:
        p_kx = p_k(m, 2*np.pi*x/N)
        denom = lambd0*(1 -  p_kx*p_kx ) + mu

        numer_x = np.sin( (2*M+1)*np.pi*l0*x/N )
        denom_x = np.sin( np.pi*l0*x/N )

        I += 4*(numer_x*numer_x)/(denom*denom_x*denom_x)


      elif x!=y and x!=0 and y!=0:
        p_kx, p_ky = p_k(m, 2*np.pi*x/N), p_k(m, 2*np.pi*y/N)
        denom = lambd0*(1 -  p_kx*p_ky ) + mu

        numer_x = np.sin( (2*M+1)*np.pi*l0*x/N )
        numer_y = np.sin( (2*M+1)*np.pi*l0*y/N )
        denom_x = np.sin( np.pi*l0*x/N )
        denom_y = np.sin( np.pi*l0*y/N )

        I += 8*(numer_x*numer_y)/(denom*denom_x*denom_y)


      else:
        print('Error!')


I = I/(N**2)
I

100%|██████████| 13200/13200 [25:52<00:00,  8.50it/s]


1.2543143249115423

In [5]:
# parameters always fixed for Fig.5
K = 1         #
sigma = 0.5   #
lambd_p = 1   #
lambd = 1     #

mu_P = 1/I
# a0 = (K - mu_P/lambd) / (1 + K*lambd_p/sigma)
a0 = (1 - mu_P) / (1 + 1/sigma)
mu_P, a0

(0.7972483293376426, 0.06758389022078581)

In [6]:
# a_star = ( (lambd*a0) / (mu*(l0**2)) ) * (K - a0*(1 + K*lambd_p/sigma))
a_star = (a0*mu_P) / (mu*(l0)**2)
b_star = mu_P / ( lambd*(l0**2) )
a_star, b_star

(0.029933968649255623, 0.0885831477041825)

In [7]:
ms, a_stars, b_stars = [], [], []
ms.append(m)
a_stars.append(a_star)
b_stars.append(b_star)
ms, a_stars, b_stars

([1.0608203677510608], [0.029933968649255623], [0.0885831477041825])

#### $m/R_0$

In [ ]:
# mu=0.2    l0=3    R0=14.14
# ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# ms/R0 = [0.07072135785007072, 0.14144271570014144, 0.21216407355021216, 0.2828854314002829, 0.3536067892503536, 0.4243281471004243, 0.49504950495049505, 0.5657708628005658, 0.6364922206506365, 0.7072135785007072]
# a_stars = [0.033737957268526145, 0.03410692056687632, 0.037181309951474555, 0.042231866845946014, 0.04561763921357979, 0.04621673560157241, 0.04485763474307491, 0.04259352064079278, 0.040112152139938566, 0.0377564902401746]
# b_stars = [0.026620756405546873, 0.02704897651442896, 0.0309046781609992, 0.039094633243707856, 0.0488292039802635, 0.05785860707797647, 0.06534895997969528, 0.07126707077105596, 0.07586014314285995, 0.07941599150592005]


# mu=0.11   l0=4    R0=10.48
# ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# ms/R0 = [0.09541984732824427, 0.19083969465648853, 0.2862595419847328, 0.38167938931297707, 0.47709923664122134, 0.5725190839694656, 0.6679389312977099, 0.7633587786259541, 0.8587786259541984, 0.9541984732824427]
# a_stars = [0.019149915977380128, 0.02203843406579211, 0.03194480531981258, 0.041207054391224025, 0.04596505382370741, 0.04732274581842942, 0.04699855306374953, 0.0460467678900726, 0.04496480658388833, 0.04395191295506978]
# b_stars = [0.007133709178927724, 0.008402247432339483, 0.013425834093039154, 0.019995356372545393, 0.025908346240532367, 0.030521393456731733, 0.03393651131770665, 0.03643149710674941, 0.03826165916223138, 0.03961984440128284]


# mu=0.05   l0=7    R0=7.07
# ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, ]
# ms/R0 = [0.14144271570014144, 0.2828854314002829, 0.4243281471004243, 0.5657708628005658, 0.7072135785007072, 0.8486562942008486, 0.9900990099009901, 1.1315417256011315, 1.272984441301273, 1.4144271570014144]
# a_stars = [0.006762341402761832, 0.016077352299794522, 0.0275035183823552, 0.0328948968701681, 0.03401252753514658, 0.03340042058130663, 0.03233608152506397, 0.031294074973241826, 0.030409340613563332, 0.02969531608674057]
# b_stars = [0.0010705041924956636, 0.002794162781907134, 0.005739908927858858, 0.00835350926779459, 0.01026152478192986, 0.011574153350800727, 0.012470196655612928, 0.013089405405169783, 0.013525747443817973, 0.0138399107167667]

In [ ]:
(np.arange(10)+1)/7.07

array([0.14144272, 0.28288543, 0.42432815, 0.56577086, 0.70721358,
       0.84865629, 0.99009901, 1.13154173, 1.27298444, 1.41442716])

In [ ]:
ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

a_stars_0 = [0.033737957268526145, 0.03410692056687632, 0.037181309951474555, 0.042231866845946014, 0.04561763921357979, 0.04621673560157241, 0.04485763474307491, 0.04259352064079278, 0.040112152139938566, 0.0377564902401746]      # mu=0.2
a_stars_1 = [0.019149915977380128, 0.02203843406579211, 0.03194480531981258, 0.041207054391224025, 0.04596505382370741, 0.04732274581842942, 0.04699855306374953, 0.0460467678900726, 0.04496480658388833, 0.04395191295506978]          # mu=0.11
a_stars_2 = [0.006762341402761832, 0.016077352299794522, 0.0275035183823552, 0.0328948968701681, 0.03401252753514658, 0.03340042058130663, 0.03233608152506397, 0.031294074973241826, 0.030409340613563332, 0.02969531608674057]    # mu=0.05

b_stars_0 = [0.026620756405546873, 0.02704897651442896, 0.0309046781609992, 0.039094633243707856, 0.0488292039802635, 0.05785860707797647, 0.06534895997969528, 0.07126707077105596, 0.07586014314285995, 0.07941599150592005]          # mu=0.2
b_stars_1 = [0.007133709178927724, 0.008402247432339483, 0.013425834093039154, 0.019995356372545393, 0.025908346240532367, 0.030521393456731733, 0.03393651131770665, 0.03643149710674941, 0.03826165916223138, 0.03961984440128284]       # mu=0.11
b_stars_2 = [0.0010705041924956636, 0.002794162781907134, 0.005739908927858858, 0.00835350926779459, 0.01026152478192986, 0.011574153350800727, 0.012470196655612928, 0.013089405405169783, 0.013525747443817973, 0.0138399107167667]  # mu=0.05


fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=ms, y=a_stars_0, name='mu=0.2', mode='lines', line_dash='dash', marker=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=ms, y=a_stars_1, name='mu=0.11', mode='lines', line_dash='dash', marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=ms, y=a_stars_2, name='mu=0.05', mode='lines', line_dash='dash', marker=dict(color='red')), row=1, col=1)


fig.add_trace(go.Scatter(x=ms, y=b_stars_0, name='mu=0.2', mode='lines', line_dash='dash', marker=dict(color='green'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=ms, y=b_stars_1, name='mu=0.11', mode='lines', line_dash='dash', marker=dict(color='blue'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=ms, y=b_stars_2, name='mu=0.05', mode='lines', line_dash='dash', marker=dict(color='red'), showlegend=False), row=2, col=1)

fig.update_layout(title_text='Theoretical prey-predator curves', hovermode='x unified', height=1000)
fig.update_xaxes(title_text='m')
fig.update_yaxes(title_text='a* - Stationary density of Predators', row=1, col=1)
fig.update_yaxes(title_text='b* - Stationary density of Preys', range = [0, 0.135], row=2, col=1)
fig.show()

#### $m$

In [ ]:
# mu=0.2    l0=3    R0=14.142
# ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# a_stars = [0.030861824709057198, 0.024178648337500518, 0.0228918561215644, 0.022446684841168626, 0.022242102114445984, 0.022131425760282323, 0.022064858058315717, 0.0220217229851834, 0.021992182354521207, 0.0219710686822388]
# b_stars = [0.08763303632450944, 0.0939549027364611, 0.09505613797719578, 0.09543003482498942, 0.09560069190763566, 0.0956927125184945, 0.09574795802001856, 0.09578371593367285, 0.09580818606320071, 0.09582566656778355]


# mu=0.11   l0=4    R0=10.48
# ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# a_stars = [0.0435114500684297, 0.039323756284493624, 0.038479076140842665, 0.03819195941423901, 0.0380615482342956, 0.03799148804516328, 0.037949531888569886, 0.037922422056681, 0.03790389242015838, 0.03789066715564424]
# b_stars = [0.040146001480363944, 0.0441150700204981, 0.04477521920691565, 0.04499239197088049, 0.04508990851370244, 0.045142014219273864, 0.04517312482161408, 0.04519318991769654, 0.04520688786349713, 0.045216656361307006]


# mu=0.05   l0=7    R0=7.07
# ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, ]
# a_stars = [0.03225967497182654, 0.02800598696499614, 0.027053993571854954, 0.026744558409471476, 0.02660949308709719, 0.02653887276451587, 0.026497342742057867, 0.02647084112914569, 0.026452886668540402, 0.02644015432500769]
# b_stars = [0.012521229644592674, 0.014492515939914408, 0.014819808476488583, 0.014921303780696981, 0.014964927152786061, 0.014987577710579087, 0.015000847986077407, 0.015009297011365002, 0.015015012672515898, 0.015019061802098263]

In [ ]:
ms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

a_stars_0 = [0.030861824709057198, 0.024178648337500518, 0.0228918561215644, 0.022446684841168626, 0.022242102114445984, 0.022131425760282323, 0.022064858058315717, 0.0220217229851834, 0.021992182354521207, 0.0219710686822388]      # mu=0.2
a_stars_1 = [0.0435114500684297, 0.039323756284493624, 0.038479076140842665, 0.03819195941423901, 0.0380615482342956, 0.03799148804516328, 0.037949531888569886, 0.037922422056681, 0.03790389242015838, 0.03789066715564424]           # mu=0.11
a_stars_2 = [0.03225967497182654, 0.02800598696499614, 0.027053993571854954, 0.026744558409471476, 0.02660949308709719, 0.02653887276451587, 0.026497342742057867, 0.02647084112914569, 0.026452886668540402, 0.02644015432500769]      # mu=0.05

b_stars_0 = [0.08763303632450944, 0.0939549027364611, 0.09505613797719578, 0.09543003482498942, 0.09560069190763566, 0.0956927125184945, 0.09574795802001856, 0.09578371593367285, 0.09580818606320071, 0.09582566656778355]            # mu=0.2
b_stars_1 = [0.040146001480363944, 0.0441150700204981, 0.04477521920691565, 0.04499239197088049, 0.04508990851370244, 0.045142014219273864, 0.04517312482161408, 0.04519318991769654, 0.04520688786349713, 0.045216656361307006]        # mu=0.11
b_stars_2 = [0.012521229644592674, 0.014492515939914408, 0.014819808476488583, 0.014921303780696981, 0.014964927152786061, 0.014987577710579087, 0.015000847986077407, 0.015009297011365002, 0.015015012672515898, 0.015019061802098263]   # mu=0.05


fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=ms, y=a_stars_0, name='mu=0.2', mode='lines', line_dash='dash', marker=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=ms, y=a_stars_1, name='mu=0.11', mode='lines', line_dash='dash', marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=ms, y=a_stars_2, name='mu=0.05', mode='lines', line_dash='dash', marker=dict(color='red')), row=1, col=1)


fig.add_trace(go.Scatter(x=ms, y=b_stars_0, name='mu=0.2', mode='lines', line_dash='dash', marker=dict(color='green'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=ms, y=b_stars_1, name='mu=0.11', mode='lines', line_dash='dash', marker=dict(color='blue'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=ms, y=b_stars_2, name='mu=0.05', mode='lines', line_dash='dash', marker=dict(color='red'), showlegend=False), row=2, col=1)

fig.update_layout(title_text='Theoretical prey-predator curves', hovermode='x unified', height=1000)
fig.update_xaxes(title_text='m')
fig.update_yaxes(title_text='a* - Stationary density of Predators', row=1, col=1)
fig.update_yaxes(title_text='b* - Stationary density of Preys', range = [0, 0.135], row=2, col=1)
fig.show()